In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import display

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('mode.chained_assignment', None)

In [2]:
# Retourne un df comprenant les lignes du df d'entrée où au moins une valeur est vide
def valeur_vide(df):
    data = df.copy()
    
    for column in df.columns:
        data = data[data[column].astype(str).str.len() < 1]
    
    if data.shape[0] == 0 :
        print("Aucune ligne n'a de valeur vide.")
        return data
    else :
        print(f'Il y a {data.shape[0]} lignes avec des valeurs vides.')
        return data

# PREPARATION
Dans cette partie, nous allons explorer les données et mettre en avant les éventuelles abérrations.  

### Produits

In [3]:
df_produits = pd.read_csv('Sources/products.csv')
display(df_produits.head(10))

produit_prix_bas = df_produits.price.sort_values().iloc[0]
produit_prix_haut = df_produits.price.sort_values().iloc[-1]

print(f"""
Nombre de ligne: {df_produits.shape[0]}
Prix le plus bas: {produit_prix_bas}
Prix le plus haut: {produit_prix_haut}
""")

print("Lignes avec des valeurs vides:")
display(valeur_vide(df_produits))
print("")
print("Prix inférieurs à 0 :")
display(df_produits.query("price < 0"))

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0
5,0_1163,9.99,0
6,1_463,36.99,1
7,0_2157,34.99,0
8,0_1915,16.99,0
9,0_389,18.99,0



Nombre de ligne: 3287
Prix le plus bas: -1.0
Prix le plus haut: 300.0

Lignes avec des valeurs vides:
Aucune ligne n'a de valeur vide.


,id_prod,price,categ



Prix inférieurs à 0 :


,id_prod,price,categ
731,T_0,-1.0,0


### Transactions

In [4]:
df_transactions = pd.read_csv('Sources/transactions.csv')
display(df_transactions.head())

t_ancienne = df_transactions.date.sort_values().iloc[1]
t_recente = df_transactions.date.sort_values().iloc[-1]

print(f"""
Date la plus ancienne: {t_ancienne}
Date la plus récente: {t_recente}
""")

dates_erreurs = df_transactions[df_transactions.date.str.contains("test")].copy()
erreurs_session_id = dates_erreurs['session_id'].unique()
erreurs_client_id = dates_erreurs['client_id'].unique()
erreurs_id_prod = dates_erreurs['id_prod'].unique()

print(f"""
Dates érronées :
{dates_erreurs.shape[0]} lignes

Sessions ID correspondant aux dates érronées:
{erreurs_session_id}

Client ID correspondants aux dates érronées:
{erreurs_client_id}

ID Produit correspondant aux dates érronées:
{erreurs_id_prod}

Lignes avec des valeurs vides:
""")
display(valeur_vide(df_transactions))

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242



Date la plus ancienne: 2021-03-01 00:02:26.047414
Date la plus récente: test_2021-03-01 02:30:02.237450


Dates érronées :
200 lignes

Sessions ID correspondant aux dates érronées:
['s_0']

Client ID correspondants aux dates érronées:
['ct_1' 'ct_0']

ID Produit correspondant aux dates érronées:
['T_0']

Lignes avec des valeurs vides:

Aucune ligne n'a de valeur vide.


,id_prod,date,session_id,client_id


### Clients

In [5]:
df_clients = pd.read_csv('Sources/customers.csv')

display(df_clients.head(10))

clients_ages = df_clients.birth.sort_values().iloc[1]
clients_jeunes = df_clients.birth.sort_values().iloc[-1]
clients_sans_genre = df_clients.query("sex != 'm' & sex != 'f'")

print(f"Année de naissance du client le plus vieux: {clients_ages}")
print(f"Année de naissance du client le plus jeune: {clients_jeunes}")
print("")
print("Clients sans genre défini:")
display(clients_sans_genre)
print("")
print("Lignes avec des valeurs vides:")
display(valeur_vide(df_clients))

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943
5,c_415,m,1993
6,c_285,f,1967
7,c_160,f,1978
8,c_6446,m,1971
9,c_6866,m,1993


Année de naissance du client le plus vieux: 1929
Année de naissance du client le plus jeune: 2004

Clients sans genre défini:


,client_id,sex,birth



Lignes avec des valeurs vides:
Aucune ligne n'a de valeur vide.


,client_id,sex,birth


### Création du dataframe principal  
Ici on merge toutes les données dans un seul dataframe, pour mieux les corriger

In [6]:
# Merge des t_s vers les produits
df_principal = pd.merge(left=df_produits,right=df_transactions,how='outer',on='id_prod')

# Puis merge vers les clients
df_principal = pd.merge(left=df_clients,right=df_principal, how='outer',on='client_id')

# renommage des colonnes
df_principal.columns = ['client_id', 'client_sex', 'client_birth', 'product_id', 'product_price', 'product_cat', 'transaction_date', 'transaction_sess_id']

display(df_principal)

,client_id,client_sex,client_birth,product_id,product_price,product_cat,transaction_date,transaction_sess_id
0,c_4410,f,1967.0,1_385,25.99,1.0,2021-03-22 01:40:22.782925,s_9707
1,c_4410,f,1967.0,0_1110,4.71,0.0,2021-11-04 16:28:30.169021,s_114715
2,c_4410,f,1967.0,0_1111,19.99,0.0,2021-03-22 01:27:49.480137,s_9707
3,c_4410,f,1967.0,1_461,12.99,1.0,2021-08-11 08:40:47.495793,s_74236
4,c_4410,f,1967.0,1_536,11.21,1.0,2022-01-18 17:05:07.468131,s_151740
...,...,...,...,...,...,...,...,...
337054,NaN,NaN,NaN,0_525,2.99,0.0,NaN,NaN
337055,NaN,NaN,NaN,2_86,132.36,2.0,NaN,NaN
337056,NaN,NaN,NaN,0_299,22.99,0.0,NaN,NaN
337057,NaN,NaN,NaN,0_510,23.66,0.0,NaN,NaN


# Nettoyage

### Y a-t-il des valeurs NaN ?

In [7]:
# Recenssement des lignes ayant une valeur NaN dans client_id.
# On créé une liste des product_id correspondant.
produits_invendus_id = df_principal.loc[df_principal.client_id.isnull(), 'product_id'].unique()

# Recenssement des lignes ayant une valeur NaN dans product_id.
# On créé une liste des client_id correspondant.
client_sans_achat_id = df_principal.loc[df_principal.product_id.isnull(), 'client_id'].unique()

print(f'''
Nombre de clients n'ayant jamais acheté : {len(client_sans_achat_id)}
Nombre de produits n'ayant jamais été vendu : {len(produits_invendus_id)}
''')


Nombre de clients n'ayant jamais acheté : 21
Nombre de produits n'ayant jamais été vendu : 22



### Suppression des valeurs de tests

Les lignes correspondantes aux valeurs de tests :
- les clients sont ct_0 ou ct_1
- les sessions de transactions sont s_0
- les dates de transactions contiennent 'test'
- les produits achetés sont t_0 et coutent -1 euro

Nous supprimons ces lignes car elles ne correspondent à aucune ventes réelles et pourraient fausser notre analyse.



In [8]:
# Suppression des lignes avec des client_id non nul et ne contenant pas 'ct"
df_principal = df_principal.loc[~(df_principal.client_id.notnull() & (df_principal.client_id.str.contains('ct')))]

# Il apparait qu'en supprimant les lignes correspondantes aux "clients df_test_cats" ct_0 et ct_1, toutes les valeurs df_test_cats sont supprimées
df_principal.head(10)

,client_id,client_sex,client_birth,product_id,product_price,product_cat,transaction_date,transaction_sess_id
0,c_4410,f,1967.0,1_385,25.99,1.0,2021-03-22 01:40:22.782925,s_9707
1,c_4410,f,1967.0,0_1110,4.71,0.0,2021-11-04 16:28:30.169021,s_114715
2,c_4410,f,1967.0,0_1111,19.99,0.0,2021-03-22 01:27:49.480137,s_9707
3,c_4410,f,1967.0,1_461,12.99,1.0,2021-08-11 08:40:47.495793,s_74236
4,c_4410,f,1967.0,1_536,11.21,1.0,2022-01-18 17:05:07.468131,s_151740
5,c_4410,f,1967.0,1_190,14.53,1.0,2021-11-12 18:11:43.280574,s_118628
6,c_4410,f,1967.0,0_1334,17.74,0.0,2021-09-25 00:17:38.676453,s_94984
7,c_4410,f,1967.0,1_616,29.02,1.0,2021-12-01 07:31:51.359660,s_127714
8,c_4410,f,1967.0,1_558,24.51,1.0,2021-09-25 00:11:19.292740,s_94984
9,c_4410,f,1967.0,0_1376,16.24,0.0,2021-09-24 22:58:27.418343,s_94984


### Le produit 0_2245 est présent dans le dataframe des transactions mais ne l'est pas dans celui des produits
Il en résulte que dans notre dataframe principal, les lignes correspondantes à ce produit n'ont pas de prix ni de catégories.

Pour y remédier, nous allons lui attribuer la catégorie correspondant à son ID (le premier caractère de l'id produit est égale à la catégorie de ce produit).
Puis nous ferons une moyenne des prix de cette catégorie que nous attriburons à ce produit.


In [9]:
# Tout d'abord, vérifions que le premier caractère de l'id produit correspond bien à la catégorie dudit produit
df_test_cat = df_principal.loc[((df_principal.product_id != '0_2245') & df_principal.product_id.notnull())].copy()
df_test_cat['equal'] = df_test_cat['product_id'].str[0] == df_test_cat['product_cat'].astype(str).str[0]
# Le df_test_cat donne True
df_test_cat[df_test_cat['product_id'].notnull()& df_test_cat['product_cat'].notnull()].equal.all()

# Nous pouvons donc réaliser une imputation de la catégorie à partir de l'id
df_principal.loc[df_principal.product_id=='0_2245', 'product_cat'] = 0

# Puis imputer le prix par la moyenne des prix de cette catégorie
cat0_prix_moyen = df_principal.loc[df_principal.product_cat == 0, 'product_price'].mean()
df_principal.loc[df_principal.product_id=='0_2245', 'product_price'] = cat0_prix_moyen

df_principal.loc[df_principal.product_id=='0_2245', ['product_id', 'product_cat', 'product_price']].drop_duplicates()

,product_id,product_cat,product_price
2678,0_2245,0.0,10.647072


### On transforme les string dates de transactions en objets datetime

In [10]:
# fonction permettant de convertir les string des dates en objets datetime 
def toDate(val):
    if pd.isnull(val):
        return val
    else:
        return datetime.strptime(val, '%Y-%m-%d %H:%M:%S.%f')

# Application de la fonction
df_principal['transaction_date'] = df_principal['transaction_date'].apply(toDate)

### On transforme les catégories produits en string

In [11]:
# Les NaN sont remplacés par un entier afin de pouvoir les transformer en string.
# Il s'agit d'un entier négatif dont nous sommes certains qu'il n'existe pas déjà comme catégorie
df_principal['product_cat'] = df_principal['product_cat'].fillna(-1)

# Transformation en string
df_principal['product_cat'] = df_principal['product_cat'].apply(lambda x: str(int(x)))

# On revient aux valeurs originelle en replacant les -1 par des NaN
df_principal['product_cat'] = df_principal['product_cat'].replace('-1', np.nan)

### On détermine l'age des clients au moment de l'achat et on le discrétise

In [12]:
# L'age du client au moment de l'achat est calculé puis intégré au dataframe
df_principal['client_age'] = df_principal['transaction_date'].dt.year - df_principal['client_birth']

# fonction permettant de classer les ages des clients selon des tranches définis
# J'utilise la méthode des amplitudes égales pour définir les classes de discrétisation
def discretise_age(age):
    tranches = [
        {'bornes': (15,29), 'label': '-30'},
        {'bornes': (30,44), 'label': '30-44'},
        {'bornes': (45,59), 'label': '45-59'},
        {'bornes': (60,95),'label': '60+'}
    ]
    for t in tranches:
        if t['bornes'][0] <= age <= t['bornes'][1]:
            return t['label']

# Application de la fonction
df_principal['client_tranche_age'] = df_principal['client_age'].apply(discretise_age)

### On discrétise les prix

In [13]:
# fonction permettant de classer les prix des produits selon des tranches définis
# J'utilise la méthode des quantiles pour définir les classes de discrétisation
def discretise_prix(prix):
    tranches = [
        {'bornes': (0,8.999),  'label': '-9€'},
        {'bornes': (9,13.999), 'label': '9€-14€'},
        {'bornes': (14,18.999), 'label': '14€-19€'},
        {'bornes': (19,301), 'label': '+19€'}
    ]
    for t in tranches:
        if t['bornes'][0] <= prix < t['bornes'][1]:
            return t['label']

# Application de la fonction
df_principal['product_tranche_prix'] = df_principal['product_price'].apply(discretise_prix)

### On exprime la date de différentes manières

In [14]:
# année d'achat
df_principal['transaction_year'] = df_principal['transaction_date'].dt.year
# mois d'achat
df_principal['transaction_month'] = df_principal['transaction_date'].dt.month
# début, fin du mois ? 0,1,2,3
df_principal['transaction_month_part'] = (df_principal['transaction_date'].dt.day / (df_principal['transaction_date'].dt.days_in_month/4+1)).apply(lambda x: str(x)[0])
# jour de la semaine
df_principal['transaction_weekday'] = df_principal['transaction_date'].dt.dayofweek
# heure d'achat
df_principal['transaction_hour'] = df_principal['transaction_date'].dt.hour
# mois de la période
df_principal['transaction_period_month'] = 1
for i in df_principal.index:
    if df_principal['transaction_year'][i] == 2022 :
        df_principal['transaction_period_month'][i] = df_principal['transaction_month'][i] + 12
    else :
        df_principal['transaction_period_month'][i] = df_principal['transaction_month'][i]


### Le cas du mois d'Octobre:
Il apparait que la majorité des ventes correspondantes à la catégorie 1 du mois d'octobre sont manquantes.
Pour ne pas fausser les données, je décide de supprimer la totalité des ventes de la catégorie 1 du mois d'octobre.

In [15]:
df_oct_cat1 = df_principal.loc[((df_principal.product_cat == '1') & (df_principal.transaction_period_month == 10.0))]
df_principal.drop(df_oct_cat1.index, inplace=True)

### Le cas des clients spéciaux :
Il apparait que certains clients ont des paniers très importants (en nombre d'article et en montant de la dépense).  
Je décide de les "marquer" comme 'gros clients' ce qui pourra me permettre de les exclure, si besoin, de l'analyse.


In [16]:
gros_clients = df_principal.groupby('client_id').agg({
    'transaction_sess_id':'count',
    'product_price':['sum', 'count'],
    })
gros_clients = gros_clients.reset_index()
gros_clients.columns = ['client_id', 'nb_sess', 'ca', 'nb_t']

gros_clients = gros_clients.loc[gros_clients.ca > 2600, 'client_id']

df_principal['Gros_client'] = 'No'
df_principal.loc[df_principal.client_id.isin(gros_clients), 'Gros_client'] = 'Yes'

sum_gros_clients = df_principal.loc[df_principal['Gros_client'] == 'Yes', 'product_price'].sum()
print(f'Il y a {len(gros_clients)} clients abbérants. Leurs achats représentent {sum_gros_clients / df_principal.product_price.sum()*100:.0f}% du CA.')

Il y a 4 clients abbérants. Leurs achats représentent 7% du CA.


# EXPORTATION

In [17]:
display(df_principal.head(15))
df_principal.to_csv("Export/export_nettoyage.csv", index=False)

,client_id,client_sex,client_birth,product_id,product_price,product_cat,transaction_date,transaction_sess_id,client_age,client_tranche_age,product_tranche_prix,transaction_year,transaction_month,transaction_month_part,transaction_weekday,transaction_hour,transaction_period_month,Gros_client
0,c_4410,f,1967.0,1_385,25.99,1,2021-03-22 01:40:22.782925,s_9707,54.0,45-59,+19€,2021.0,3.0,2,0.0,1.0,3.0,No
1,c_4410,f,1967.0,0_1110,4.71,0,2021-11-04 16:28:30.169021,s_114715,54.0,45-59,-9€,2021.0,11.0,0,3.0,16.0,11.0,No
2,c_4410,f,1967.0,0_1111,19.99,0,2021-03-22 01:27:49.480137,s_9707,54.0,45-59,+19€,2021.0,3.0,2,0.0,1.0,3.0,No
3,c_4410,f,1967.0,1_461,12.99,1,2021-08-11 08:40:47.495793,s_74236,54.0,45-59,9€-14€,2021.0,8.0,1,2.0,8.0,8.0,No
4,c_4410,f,1967.0,1_536,11.21,1,2022-01-18 17:05:07.468131,s_151740,55.0,45-59,9€-14€,2022.0,1.0,2,1.0,17.0,13.0,No
5,c_4410,f,1967.0,1_190,14.53,1,2021-11-12 18:11:43.280574,s_118628,54.0,45-59,14€-19€,2021.0,11.0,1,4.0,18.0,11.0,No
6,c_4410,f,1967.0,0_1334,17.74,0,2021-09-25 00:17:38.676453,s_94984,54.0,45-59,14€-19€,2021.0,9.0,2,5.0,0.0,9.0,No
7,c_4410,f,1967.0,1_616,29.02,1,2021-12-01 07:31:51.359660,s_127714,54.0,45-59,+19€,2021.0,12.0,0,2.0,7.0,12.0,No
8,c_4410,f,1967.0,1_558,24.51,1,2021-09-25 00:11:19.292740,s_94984,54.0,45-59,+19€,2021.0,9.0,2,5.0,0.0,9.0,No
9,c_4410,f,1967.0,0_1376,16.24,0,2021-09-24 22:58:27.418343,s_94984,54.0,45-59,14€-19€,2021.0,9.0,2,4.0,22.0,9.0,No
